# Wikipedia training

In this tutorial we will:
 - Learn how to train the NMF topic model on English Wikipedia corpus
 - Compare it with LDA model
 - Evaluate results

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import json
import logging
import numpy as np
import pandas as pd
import scipy.sparse
import smart_open
import time
import os
import psutil
from contextlib import contextmanager
from multiprocessing import Process
from tqdm import tqdm, tqdm_notebook
import joblib

import gensim.downloader as api
from gensim import matutils
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.nmf import Nmf as GensimNmf
from sklearn.decomposition.nmf import NMF as SklearnNmf
from gensim.parsing.preprocessing import preprocess_string

tqdm.pandas()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

# Preprocessing

### Load wikipedia dump
Let's use `gensim.downloader.api` for that

In [2]:
data = api.load("wiki-english-20171001")
article = next(iter(data))

for section_title, section_text in zip(
    article['section_titles'],
    article['section_texts']
):
    print("Section title: %s" % section_title)
    print("Section text: %s" % section_text[:100])

Section title: Introduction
Section text: 




'''Anarchism''' is a political philosophy that advocates self-governed societies based on volun
Section title: Etymology and terminology
Section text: 

The word ''anarchism'' is composed from the word ''anarchy'' and the suffix ''-ism'', themselves d
Section title: History
Section text: 

===Origins===
Woodcut from a Diggers document by William Everard

The earliest anarchist themes ca
Section title: Anarchist schools of thought
Section text: 
Portrait of philosopher Pierre-Joseph Proudhon (1809–1865) by Gustave Courbet. Proudhon was the pri
Section title: Internal issues and debates
Section text: 
consistent with anarchist values is a controversial subject among anarchists.

Anarchism is a philo
Section title: Topics of interest
Section text: Intersecting and overlapping between various schools of thought, certain topics of interest and inte
Section title: Criticisms
Section text: 
Criticisms of anarchism include moral criticisms and pra

Preprocess and save articles

In [3]:
def save_preprocessed_articles(filename, articles):
    with smart_open(filename, 'w+', encoding="utf8") as writer:
        for article in tqdm_notebook(articles):
            article_text = " ".join(
                " ".join(section)
                for section
                in zip(
                    article['section_titles'],
                    article['section_texts']
                )
            )
            article_text = preprocess_string(article_text)

            writer.write(json.dumps(article_text) + '\n')


def get_preprocessed_articles(filename):
    with smart_open(filename, 'r', encoding="utf8") as reader:
        for line in tqdm_notebook(reader):
            yield json.loads(
                line
            )

In [4]:
SAVE_ARTICLES = False

if SAVE_ARTICLES:
    save_preprocessed_articles('wiki_articles.jsonlines', data)

### Create and save dictionary

In [5]:
SAVE_DICTIONARY = False

if SAVE_DICTIONARY:
    dictionary = Dictionary(get_preprocessed_articles('wiki_articles.jsonlines'))
    dictionary.save('wiki.dict')

### Load and filter dictionary

In [6]:
dictionary = Dictionary.load('wiki.dict')
dictionary.filter_extremes()
dictionary.compactify()

2019-01-30 23:49:27,738 : INFO : loading Dictionary object from wiki.dict
2019-01-30 23:49:28,637 : INFO : loaded wiki.dict
2019-01-30 23:49:33,783 : INFO : discarding 1910146 tokens: [('abdelrahim', 49), ('abstention', 120), ('anarcha', 101), ('anarchica', 40), ('anarchosyndicalist', 20), ('antimilitar', 68), ('arbet', 194), ('archo', 100), ('arkhē', 5), ('autonomedia', 118)]...
2019-01-30 23:49:33,784 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 2462447 (=50.0%) documents
2019-01-30 23:49:34,701 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['omana', 'thoroughfar', 'janssen', 'boletacea', 'itzik']...)


### MmCorpus wrapper
In this way we'll:

- Make sure that documents are shuffled
- Be able to train-test split corpus without rewriting it

In [7]:
class RandomCorpus(MmCorpus):
    def __init__(self, random_seed=42, testset=False, testsize=1000, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)

        random_state = np.random.RandomState(random_seed)
        
        self.indices = random_state.permutation(range(self.num_docs))
        test_nnz = sum(len(self[doc_idx]) for doc_idx in self.indices[:testsize])
        
        if testset:
            self.indices = self.indices[:testsize]
            self.num_docs = testsize
            self.num_nnz = test_nnz
        else:
            self.indices = self.indices[testsize:]
            self.num_docs -= testsize
            self.num_nnz -= test_nnz

    def __iter__(self):
        for doc_id in self.indices:
            yield self[doc_id]

### Create and save corpus

In [8]:
SAVE_CORPUS = False

if SAVE_CORPUS:
    corpus = (
        dictionary.doc2bow(article)
        for article
        in get_preprocessed_articles('wiki_articles.jsonlines')
    )
    
    RandomCorpus.serialize('wiki.mm', corpus)

### Load train and test corpus
Using `RandomCorpus` wrapper

In [9]:
train_corpus = RandomCorpus(
    random_seed=42, testset=False, testsize=2000, fname='wiki.mm'
)
test_corpus = RandomCorpus(
    random_seed=42, testset=True, testsize=2000, fname='wiki.mm'
)

2019-01-30 23:49:35,606 : INFO : loaded corpus index from wiki.mm.index
2019-01-30 23:49:35,607 : INFO : initializing cython corpus reader from wiki.mm
2019-01-30 23:49:35,607 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries
2019-01-30 23:49:37,629 : INFO : loaded corpus index from wiki.mm.index
2019-01-30 23:49:37,630 : INFO : initializing cython corpus reader from wiki.mm
2019-01-30 23:49:37,630 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries


### Convert corpora to csc and save

In [10]:
SAVE_CSC = False

if SAVE_CSC:
    train_csc = matutils.corpus2csc(train_corpus, len(dictionary))
    scipy.sparse.save_npz('train_csc.npz', train_csc)
    
    test_csc = matutils.corpus2csc(test_corpus, len(dictionary))
    scipy.sparse.save_npz('test_csc.npz', test_csc)

### Load csc

In [11]:
train_csc = scipy.sparse.load_npz('train_csc.npz')
test_csc = scipy.sparse.load_npz('test_csc.npz')

## Metrics

In [12]:
@contextmanager
def measure_ram(output, tick=2):
    def _measure_ram(pid, output, tick=5):
        py = psutil.Process(pid)
        with open(output, 'w') as outfile:
            while True:
                memory = py.memory_info().rss
                outfile.write("{}\n".format(memory))
                outfile.flush()
                time.sleep(tick)

    pid = os.getpid()
    p = Process(target=_measure_ram, args=(pid, output, tick))
    p.start()
    yield
    p.terminate()

def get_train_time_and_ram(func, name):
    memprof_filename = "{}.memprof".format(name)
    
    start = time.time()

    with measure_ram(memprof_filename, 5):
        result = func()        
        
    elapsed_time = pd.to_timedelta(time.time() - start, unit='s').round('s')
    
    memprof_df = pd.read_csv(memprof_filename, squeeze=True)
    
    mean_ram = "{} MB".format(
        memprof_df.mean() // 2**20,
    )
    
    max_ram = "{} MB".format(memprof_df.max() // 2**20)

    return elapsed_time, mean_ram, max_ram, result


def get_tm_metrics(model, test_corpus):
    W = model.get_topics().T
    H = np.zeros((model.num_topics, len(test_corpus)))
    for bow_id, bow in enumerate(test_corpus):
        for topic_id, word_count in model.get_document_topics(bow):
            H[topic_id, bow_id] = word_count

    pred_factors = W.dot(H)
    pred_factors /= pred_factors.sum(axis=0)
    
    dense_corpus = matutils.corpus2dense(test_corpus, pred_factors.shape[0])

    perplexity = get_tm_perplexity(pred_factors, dense_corpus)

    l2_norm = get_tm_l2_norm(pred_factors, dense_corpus)

    model.normalize = True

    coherence = CoherenceModel(
        model=model,
        corpus=test_corpus,
        coherence='u_mass'
    ).get_coherence()

    model.normalize = False

    return dict(
        perplexity=round(perplexity, 4),
        coherence=round(coherence, 4),
        l2_norm=round(l2_norm, 4),
    )


def get_tm_perplexity(pred_factors, dense_corpus):
    return np.exp(-(np.log(pred_factors, where=pred_factors > 0) * dense_corpus).sum() / dense_corpus.sum())


def get_tm_l2_norm(pred_factors, dense_corpus):
    return np.linalg.norm(dense_corpus / dense_corpus.sum(axis=0) - pred_factors)


def get_sklearn_metrics(model, test_corpus):
    W = model.components_.T
    H = model.transform((test_corpus / test_corpus.sum(axis=0)).T).T
    pred_factors = W.dot(H)
    pred_factors /= pred_factors.sum(axis=0)

    perplexity = np.exp(
        -(np.log(pred_factors, where=pred_factors > 0) * test_corpus).sum()
        / test_corpus.sum()
    )

    l2_norm = np.linalg.norm(test_corpus / test_corpus.sum(axis=0) - pred_factors)

    return dict(
        perplexity=perplexity,
        l2_norm=l2_norm,
    )

Define dataframe in which we'll store metrics

In [13]:
tm_metrics = pd.DataFrame()

### Define common params for the models

In [14]:
params = dict(
    corpus=train_corpus,
    chunksize=2000,
    num_topics=50,
    id2word=dictionary,
    passes=1,
    eval_every=10,
    minimum_probability=0,
    random_state=42,
)

## Training

### Train Gensim NMF and save it
Normalization is turned off to compute metrics correctly

In [15]:
row = dict()
row['model'] = 'gensim_nmf'
row['train_time'], row['mean_ram'], row['max_ram'], nmf = get_train_time_and_ram(
    lambda: GensimNmf(
        normalize=False,
        **params
    ),
    'gensim_nmf',
)

nmf.save('gensim_nmf.model')

2019-01-30 23:50:41,538 : INFO : Loss: 0.9592283350028137
2019-01-30 23:51:07,841 : INFO : Loss: 0.9654052723463945
2019-01-30 23:51:19,986 : INFO : Loss: 0.983610134773025
2019-01-30 23:51:31,494 : INFO : Loss: 0.9856176961379628
2019-01-30 23:51:47,703 : INFO : Loss: 1.0388382070148916
2019-01-30 23:52:00,235 : INFO : Loss: 0.9885831160386329
2019-01-30 23:52:10,915 : INFO : Loss: 0.9940857027761342
2019-01-30 23:52:19,815 : INFO : Loss: 0.9946127359838275
2019-01-30 23:52:28,330 : INFO : Loss: 0.9949275215480511
2019-01-30 23:52:36,515 : INFO : Loss: 0.9957360140573942
2019-01-30 23:52:45,287 : INFO : Loss: 0.995799267839182
2019-01-30 23:52:53,664 : INFO : Loss: 0.9950670862672046
2019-01-30 23:53:00,937 : INFO : Loss: 0.9959924745279174
2019-01-30 23:53:09,471 : INFO : Loss: 0.9959557971255888
2019-01-30 23:53:17,299 : INFO : Loss: 0.9992069754012614
2019-01-30 23:53:24,887 : INFO : Loss: 0.9970817907880735
2019-01-30 23:53:31,921 : INFO : Loss: 0.9968030668126687
2019-01-30 23:53

2019-01-31 00:05:47,007 : INFO : Loss: 0.9996352155288082
2019-01-31 00:05:52,261 : INFO : Loss: 1.0004395613185166
2019-01-31 00:05:57,342 : INFO : Loss: 1.0
2019-01-31 00:06:02,548 : INFO : Loss: 0.9998527076273703
2019-01-31 00:06:07,609 : INFO : Loss: 0.9996924850765493
2019-01-31 00:06:12,841 : INFO : Loss: 0.9998465986185003
2019-01-31 00:06:18,157 : INFO : Loss: 0.9997732510121399
2019-01-31 00:06:23,211 : INFO : Loss: 0.9997941508395527
2019-01-31 00:06:28,233 : INFO : Loss: 0.9995770839553472
2019-01-31 00:06:33,449 : INFO : Loss: 0.9997390770111323
2019-01-31 00:06:38,821 : INFO : Loss: 0.9997499890547042
2019-01-31 00:06:43,967 : INFO : Loss: 1.0002122989122686
2019-01-31 00:06:48,974 : INFO : Loss: 1.0
2019-01-31 00:06:54,205 : INFO : Loss: 0.9996768801524232
2019-01-31 00:06:59,422 : INFO : Loss: 0.9998134505747993
2019-01-31 00:07:04,626 : INFO : Loss: 1.0001153274263432
2019-01-31 00:07:09,638 : INFO : Loss: 0.9997996517667606
2019-01-31 00:07:14,849 : INFO : Loss: 0.999

### Load Gensim NMF and store metrics

In [16]:
nmf = GensimNmf.load('gensim_nmf.model')
row.update(get_tm_metrics(nmf, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-01-31 00:14:19,539 : INFO : loading Nmf object from gensim_nmf.model
2019-01-31 00:14:19,856 : INFO : loading id2word recursively from gensim_nmf.model.id2word.* with mmap=None
2019-01-31 00:14:19,856 : INFO : loaded gensim_nmf.model
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-01-31 00:14:59,973 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-01-31 00:15:00,082 : INFO : CorpusAccumulator accumulated stats from 2000 documents


### Train LDA and save it
That's a common model to do Topic Modeling

In [17]:
row = dict()
row['model'] = 'lda'
row['train_time'], row['mean_ram'], row['max_ram'], lda = get_train_time_and_ram(
    lambda: LdaModel(**params),
    'lda',
)
lda.save('lda.model')

2019-01-31 00:15:00,190 : INFO : using symmetric alpha at 0.02
2019-01-31 00:15:00,192 : INFO : using symmetric eta at 0.02
2019-01-31 00:15:00,209 : INFO : using serial LDA version on this node
2019-01-31 00:15:00,734 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 4922894 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-01-31 00:15:00,890 : INFO : PROGRESS: pass 0, at document #2000/4922894
2019-01-31 00:15:02,814 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 00:15:03,134 : INFO : topic #36 (0.020): 0.006*"new" + 0.005*"reconstruct" + 0.004*"serv" + 0.004*"includ" + 0.003*"yawn" + 0.003*"start" + 0.003*"depress" + 0.003*"word" + 0.003*"théori" + 0.003*"american"
2019-01-31 00:15:03,136 : INFO : topic #19 (0.020): 0.005*"taxpay" + 0.005*"new" + 0.004*"nation" + 0.004*"start" + 0.004*

2019-01-31 00:15:14,199 : INFO : topic diff=0.220294, rho=0.408248
2019-01-31 00:15:14,358 : INFO : PROGRESS: pass 0, at document #14000/4922894
2019-01-31 00:15:16,536 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 00:15:16,796 : INFO : topic #39 (0.020): 0.038*"taxpay" + 0.025*"scientist" + 0.025*"clot" + 0.020*"leagu" + 0.015*"place" + 0.012*"player" + 0.012*"folei" + 0.010*"hoar" + 0.009*"yawn" + 0.009*"basketbal"
2019-01-31 00:15:16,797 : INFO : topic #0 (0.020): 0.035*"statewid" + 0.031*"arsen" + 0.015*"line" + 0.014*"raid" + 0.012*"museo" + 0.012*"pain" + 0.012*"left" + 0.011*"alic" + 0.010*"word" + 0.009*"artist"
2019-01-31 00:15:16,798 : INFO : topic #13 (0.020): 0.020*"sourc" + 0.018*"north" + 0.015*"weekli" + 0.013*"earthworm" + 0.012*"castl" + 0.012*"lagrang" + 0.010*"cotton" + 0.009*"vigour" + 0.008*"vacant" + 0.008*"hormon"
2019-01-31 00:15:16,800 : INFO : topic #45 (0.020): 0.015*"depress" + 0.007*"slow" + 0.006*"stanc" + 0.006*

2019-01-31 00:15:29,649 : INFO : PROGRESS: pass 0, at document #26000/4922894
2019-01-31 00:15:31,187 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 00:15:31,448 : INFO : topic #10 (0.020): 0.010*"cdd" + 0.009*"disco" + 0.007*"cancer" + 0.007*"gastrointestin" + 0.007*"acid" + 0.007*"caus" + 0.006*"pathwai" + 0.006*"treat" + 0.006*"proper" + 0.005*"includ"
2019-01-31 00:15:31,449 : INFO : topic #18 (0.020): 0.007*"man" + 0.006*"kill" + 0.005*"charact" + 0.005*"théori" + 0.004*"septemb" + 0.004*"later" + 0.004*"appear" + 0.004*"epiru" + 0.004*"storag" + 0.004*"faster"
2019-01-31 00:15:31,450 : INFO : topic #0 (0.020): 0.055*"statewid" + 0.038*"arsen" + 0.024*"line" + 0.024*"raid" + 0.016*"pain" + 0.016*"word" + 0.013*"alic" + 0.012*"traceabl" + 0.012*"museo" + 0.012*"london"
2019-01-31 00:15:31,451 : INFO : topic #37 (0.020): 0.007*"love" + 0.007*"théori" + 0.006*"place" + 0.006*"rel" + 0.005*"appear" + 0.005*"night" + 0.004*"man" + 0.004*"son" 

2019-01-31 00:15:43,296 : INFO : topic #40 (0.020): 0.072*"unit" + 0.025*"collector" + 0.015*"start" + 0.014*"american" + 0.011*"new" + 0.011*"institut" + 0.009*"scholar" + 0.009*"word" + 0.009*"governor" + 0.008*"professor"
2019-01-31 00:15:43,297 : INFO : topic #25 (0.020): 0.016*"mount" + 0.015*"ring" + 0.015*"palmer" + 0.014*"lagrang" + 0.013*"mound" + 0.011*"area" + 0.008*"pcb" + 0.007*"robespierr" + 0.006*"natur" + 0.006*"surrend"
2019-01-31 00:15:43,299 : INFO : topic #27 (0.020): 0.045*"questionnair" + 0.015*"dai" + 0.012*"taxpay" + 0.012*"tornado" + 0.012*"théori" + 0.010*"rick" + 0.010*"horac" + 0.010*"squatter" + 0.010*"find" + 0.009*"sebastien"
2019-01-31 00:15:43,301 : INFO : topic #48 (0.020): 0.066*"octob" + 0.062*"januari" + 0.061*"march" + 0.057*"sens" + 0.055*"notion" + 0.053*"judici" + 0.053*"april" + 0.052*"juli" + 0.051*"august" + 0.051*"decatur"
2019-01-31 00:15:43,307 : INFO : topic diff=0.286263, rho=0.229416
2019-01-31 00:15:46,145 : INFO : -11.672 per-word bou

2019-01-31 00:15:57,749 : INFO : topic #48 (0.020): 0.065*"octob" + 0.063*"notion" + 0.063*"januari" + 0.063*"sens" + 0.060*"judici" + 0.060*"march" + 0.059*"april" + 0.059*"august" + 0.057*"decatur" + 0.056*"juli"
2019-01-31 00:15:57,751 : INFO : topic #21 (0.020): 0.025*"samford" + 0.021*"spain" + 0.019*"mexico" + 0.016*"del" + 0.012*"juan" + 0.012*"soviet" + 0.012*"mexican" + 0.011*"plung" + 0.010*"santa" + 0.010*"josé"
2019-01-31 00:15:57,752 : INFO : topic #49 (0.020): 0.028*"india" + 0.022*"incumb" + 0.007*"singh" + 0.006*"peopl" + 0.006*"pakistan" + 0.006*"treeless" + 0.006*"alam" + 0.006*"pradesh" + 0.006*"area" + 0.006*"khalsa"
2019-01-31 00:15:57,758 : INFO : topic diff=0.237375, rho=0.200000
2019-01-31 00:15:57,910 : INFO : PROGRESS: pass 0, at document #52000/4922894
2019-01-31 00:15:59,443 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 00:15:59,707 : INFO : topic #21 (0.020): 0.027*"samford" + 0.020*"spain" + 0.019*"mexico" + 0.01

...

2019-01-31 01:40:03,969 : INFO : topic diff=0.002540, rho=0.020261
2019-01-31 01:40:04,123 : INFO : PROGRESS: pass 0, at document #4874000/4922894
2019-01-31 01:40:05,481 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 01:40:05,747 : INFO : topic #22 (0.020): 0.034*"spars" + 0.017*"factor" + 0.012*"plaisir" + 0.010*"genu" + 0.010*"western" + 0.008*"biom" + 0.008*"median" + 0.007*"trap" + 0.006*"incom" + 0.006*"florida"
2019-01-31 01:40:05,749 : INFO : topic #30 (0.020): 0.035*"leagu" + 0.035*"cleveland" + 0.032*"place" + 0.027*"taxpay" + 0.024*"scientist" + 0.023*"crete" + 0.021*"folei" + 0.017*"goal" + 0.015*"martin" + 0.012*"schmitz"
2019-01-31 01:40:05,750 : INFO : topic #5 (0.020): 0.038*"abroad" + 0.028*"son" + 0.027*"rel" + 0.025*"reconstruct" + 0.021*"band" + 0.016*"muscl" + 0.016*"simultan" + 0.014*"charcoal" + 0.013*"toyota" + 0.010*"myspac"
2019-01-31 01:40:05,751 : INFO : topic #3 (0.020): 0.032*"present" + 0.025*"nation" + 0.024*"of

2019-01-31 01:40:17,265 : INFO : topic diff=0.002712, rho=0.020236
2019-01-31 01:40:17,421 : INFO : PROGRESS: pass 0, at document #4886000/4922894
2019-01-31 01:40:18,792 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-01-31 01:40:19,059 : INFO : topic #23 (0.020): 0.135*"audit" + 0.071*"best" + 0.035*"yawn" + 0.028*"jacksonvil" + 0.023*"japanes" + 0.020*"noll" + 0.018*"festiv" + 0.018*"intern" + 0.018*"women" + 0.013*"prison"
2019-01-31 01:40:19,060 : INFO : topic #8 (0.020): 0.026*"law" + 0.023*"cortic" + 0.018*"start" + 0.016*"act" + 0.012*"ricardo" + 0.012*"case" + 0.010*"replac" + 0.010*"polaris" + 0.008*"legal" + 0.008*"order"
2019-01-31 01:40:19,061 : INFO : topic #1 (0.020): 0.053*"china" + 0.043*"chilton" + 0.025*"hong" + 0.025*"kong" + 0.021*"korea" + 0.018*"korean" + 0.017*"leah" + 0.016*"shirin" + 0.016*"sourc" + 0.014*"kim"
2019-01-31 01:40:19,062 : INFO : topic #32 (0.020): 0.051*"district" + 0.046*"popolo" + 0.043*"vigour" + 0.035*"tor

2019-01-31 01:40:30,034 : INFO : topic #4 (0.020): 0.019*"enfranchis" + 0.015*"depress" + 0.013*"pour" + 0.008*"veget" + 0.008*"elabor" + 0.008*"mode" + 0.008*"uruguayan" + 0.006*"develop" + 0.006*"turn" + 0.006*"produc"
2019-01-31 01:40:30,035 : INFO : topic #26 (0.020): 0.028*"champion" + 0.028*"workplac" + 0.025*"woman" + 0.025*"olymp" + 0.023*"men" + 0.021*"alic" + 0.021*"medal" + 0.020*"event" + 0.018*"atheist" + 0.018*"taxpay"
2019-01-31 01:40:30,036 : INFO : topic #6 (0.020): 0.068*"fewer" + 0.024*"epiru" + 0.021*"septemb" + 0.018*"teacher" + 0.014*"stake" + 0.012*"rodríguez" + 0.012*"proclaim" + 0.010*"direct" + 0.010*"acrimoni" + 0.010*"movi"
2019-01-31 01:40:30,037 : INFO : topic #28 (0.020): 0.037*"build" + 0.031*"hous" + 0.019*"buford" + 0.015*"histor" + 0.012*"linear" + 0.011*"silicon" + 0.011*"centuri" + 0.011*"depress" + 0.011*"constitut" + 0.010*"pistol"
2019-01-31 01:40:30,038 : INFO : topic #16 (0.020): 0.055*"king" + 0.030*"priest" + 0.020*"duke" + 0.019*"rotterdam" 

2019-01-31 01:40:43,261 : INFO : topic #4 (0.020): 0.019*"enfranchis" + 0.015*"depress" + 0.013*"pour" + 0.008*"mode" + 0.008*"veget" + 0.008*"elabor" + 0.008*"uruguayan" + 0.006*"develop" + 0.006*"turn" + 0.006*"produc"
2019-01-31 01:40:43,262 : INFO : topic #15 (0.020): 0.011*"small" + 0.010*"organ" + 0.009*"develop" + 0.009*"commun" + 0.008*"word" + 0.008*"peopl" + 0.008*"group" + 0.007*"woman" + 0.007*"human" + 0.006*"summerhil"
2019-01-31 01:40:43,263 : INFO : topic #42 (0.020): 0.049*"german" + 0.034*"germani" + 0.015*"vol" + 0.014*"berlin" + 0.013*"israel" + 0.013*"der" + 0.013*"jewish" + 0.010*"austria" + 0.009*"europ" + 0.009*"european"
2019-01-31 01:40:43,264 : INFO : topic #6 (0.020): 0.068*"fewer" + 0.024*"epiru" + 0.021*"septemb" + 0.018*"teacher" + 0.014*"stake" + 0.012*"rodríguez" + 0.012*"proclaim" + 0.010*"direct" + 0.010*"movi" + 0.010*"acrimoni"
2019-01-31 01:40:43,265 : INFO : topic #12 (0.020): 0.009*"number" + 0.007*"frontal" + 0.007*"gener" + 0.006*"poet" + 0.006

2019-01-31 01:40:56,692 : INFO : topic #5 (0.020): 0.038*"abroad" + 0.028*"son" + 0.027*"rel" + 0.026*"reconstruct" + 0.021*"band" + 0.016*"muscl" + 0.016*"simultan" + 0.014*"charcoal" + 0.013*"toyota" + 0.010*"vocabulari"
2019-01-31 01:40:56,693 : INFO : topic #0 (0.020): 0.061*"statewid" + 0.038*"line" + 0.030*"rivièr" + 0.030*"raid" + 0.025*"rosenwald" + 0.020*"airmen" + 0.017*"traceabl" + 0.017*"serv" + 0.013*"briarwood" + 0.013*"oper"
2019-01-31 01:40:56,694 : INFO : topic #24 (0.020): 0.041*"book" + 0.036*"publicis" + 0.024*"word" + 0.020*"new" + 0.014*"edit" + 0.014*"presid" + 0.011*"magazin" + 0.011*"author" + 0.011*"nicola" + 0.011*"storag"
2019-01-31 01:40:56,695 : INFO : topic #48 (0.020): 0.077*"octob" + 0.076*"march" + 0.075*"sens" + 0.067*"januari" + 0.067*"notion" + 0.067*"decatur" + 0.066*"juli" + 0.066*"august" + 0.065*"april" + 0.063*"judici"
2019-01-31 01:40:56,696 : INFO : topic #38 (0.020): 0.022*"walter" + 0.011*"aza" + 0.009*"forc" + 0.008*"teufel" + 0.008*"batta

### Load LDA and store metrics

In [18]:
lda = LdaModel.load('lda.model')
row.update(get_tm_metrics(lda, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-01-31 01:40:59,377 : INFO : loading LdaModel object from lda.model
2019-01-31 01:40:59,379 : INFO : loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
2019-01-31 01:40:59,383 : INFO : setting ignored attribute dispatcher to None
2019-01-31 01:40:59,384 : INFO : setting ignored attribute state to None
2019-01-31 01:40:59,384 : INFO : setting ignored attribute id2word to None
2019-01-31 01:40:59,385 : INFO : loaded lda.model
2019-01-31 01:40:59,385 : INFO : loading LdaState object from lda.model.state
2019-01-31 01:40:59,479 : INFO : loaded lda.model.state
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-01-31 01:41:07,761 : INFO : CorpusAccumulator accumulated stats from 100

### Train Sklearn NMF and store metrics

In [19]:
# Normalize the input corpus to pass to sklearn
train_csc.data /= np.repeat(np.array(train_csc.sum(axis=0)), train_csc.getnnz(axis=0))

row = dict()
row['model'] = 'sklearn_nmf'
sklearn_nmf = SklearnNmf(n_components=50, tol=1e-2, random_state=42)
row['train_time'], row['mean_ram'], row['max_ram'], sklearn_nmf = get_train_time_and_ram(
    lambda: sklearn_nmf.fit(train_csc.T),
    'slearn_nmf',
)

joblib.dump(sklearn_nmf, 'sklearn_nmf.joblib')

['sklearn_nmf.joblib']

### Load Sklearn NMF and store metrics

In [20]:
sklearn_nmf = joblib.load('sklearn_nmf.joblib')
row.update(get_sklearn_metrics(
    sklearn_nmf, test_csc.toarray(),
))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

In [21]:
tm_metrics.replace(np.nan, '-', inplace=True)

## Results

In [22]:
tm_metrics

,coherence,l2_norm,max_ram,mean_ram,model,perplexity,train_time
0,-2.9077,7.338000,8587 MB,8553.0 MB,gensim_nmf,3817.239000,00:24:16
1,-2.5286,7.364800,8773 MB,8773.0 MB,lda,4701.976000,01:25:59
2,-,6.977583,17238 MB,11675.0 MB,sklearn_nmf,4437.417315,00:40:41


As we can see, NMF can be significantly faster than LDA without sacrificing quality of topics too much (or not sacrificing at all)

Moreover, NMF can be very flexible on RAM usage due to sparsity option, which leaves only small amount of elements in inner matrices.